### Import Libraries 

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from skopt import gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from skopt.space import Real, Integer
import time

### Load the dataset

In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv("heart_cleveland_upload.csv")

# Display the first few rows
print(data.head())


## Preprocess

Since the dataset contains only numerical features, we need to scale the numerical features using StandardScaler from sklearn library. We also need to split the dataset into input features (X) and output feature (y), where y is the "condition" column.

In [2]:
num_cols = data.select_dtypes(include="number").columns.tolist()
scaler = StandardScaler()
data[num_cols] = scaler.fit_transform(data[num_cols])
X = data.drop("condition", axis=1)
y = data["condition"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Neural network architecture

We can experiment with different architectures and hyperparameters. Here we use a neural network architecture with 3 hidden layers, each with 64 neurons, and dropout regularization:


In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


### fitness function
The fitness function is the accuracy of the neural network on the validation set. We can use the validation_split argument in model.fit() to automatically split the training set into training and validation sets. 

In [ ]:
def fitness_function(params):
    # Set the hyperparameters
    lr, batch_size, epochs = params
    
    model.optimizer.lr = lr

    # Train the model with the given hyperparameters
    history = model.fit(X_train, y_train, validation_split=0.2, batch_size=batch_size, epochs=epochs, verbose=0)

    # Evaluate the model on the validation set
    score = model.evaluate(X_train, y_train, verbose=0)
    accuracy = score[1]

    # Return the accuracy as the fitness value
      return -accuracy  # minimize the negative accuracy

### Define the search space for each randomized search algorithm

In [ ]:
from skopt import gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from skopt.space import Real, Integer

# Define the search space for each algorithm
space_gp = [Real(1e-5, 1e-1, "log-uniform", name="lr"), Integer(16, 128, name="batch_size"), Integer(10, 100, name="epochs")]

space_forest = [Real(1e-5, 1e-1, "log-uniform", name="lr"), Integer(16, 128, name="batch_size"), Integer(10, 100, name="epochs")]

space_gbrt = [Real(1e-5, 1e-1, "log-uniform", name="lr"), Integer(16, 128, name="batch_size"), Integer(10, 100, name="epochs")]

space_dummy = [Real(1e-5, 1e-1, "log-uniform", name="lr"), Integer(16, 128, name="batch_size"), Integer(10, 100, name="epochs")]


### Define the algorithms

In [ ]:
algorithms = [
    {"name": "GP", "function": gp_minimize, "space": space_gp},
    {"name": "Forest", "function": forest_minimize, "space": space_forest},
    {"name": "GBRT", "function": gbrt_minimize, "space": space_gbrt},
    {"name": "Dummy", "function": dummy_minimize, "space": space_dummy},
]


### Run the algorithms and print the results

In [ ]:
for algorithm in algorithms:
    print(f"Running {algorithm['name']} algorithm...")
    start_time = time.time()
    result = algorithm["function"](fitness_function, algorithm["space"], n_calls=50, random_state=42)
    end_time = time.time()
    print(f"Time taken: {end_time - start_time:.2f} seconds")
print(f"Best parameters: {result.x}")
print(f"Best fitness value: {-result.fun}\n")